In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit


In [2]:
train_1 = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\AibHack\data\training.txt", sep="\t", encoding = 'iso-8859-1')
test_1 = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\AibHack\data\testing.txt", sep="\t", encoding = 'iso-8859-1')

In [3]:
def is_outlier(points, thresh = 3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [4]:
df_train = df_train[pd.notnull(df_train['EnergyRatingCat'])]

NameError: name 'df_train' is not defined

In [5]:
df_train = train_1.ix[:, train_1.columns != 'EnergyRatingCat']

In [6]:
df = df_train.append(test_1, ignore_index = True)

In [7]:
df.head()

,AreaAperture,AreaFirstFloor,AreaGroundFloor,AreaSecondFloor,AssessmentDate,AvgDoorU,AvgFloorU,AvgRoofU,AvgWallU,AvgWindowU,...,TotWindowArea,Type,Underfloor,UpgradeHES,VentMethod,VolWaterStorage,WallStructure,WarmAir,Year,ZeroLossCollEff
0,NaN,0.00,33.69,0.0,Jun 17 2011 12:00AM,3.00,0.61,2.30,2.10,2.71,...,7.08,Mid-terrace house,NO,0.0,Natural vent.,110.0,Masonry,NO,Y1890.0,NaN
1,NaN,50.67,96.77,0.0,Apr 14 2015 12:00AM,3.96,0.94,0.75,2.01,2.63,...,37.00,Detached house,NO,0.0,Natural vent.,95.0,Masonry,NO,Y1901.0,NaN
2,NaN,0.00,104.84,0.0,Jun 28 2012 12:00AM,0.00,0.68,0.13,0.38,2.81,...,17.90,Detached house,NO,0.0,Natural vent.,110.0,Masonry,NO,Y1968.0,NaN
3,NaN,42.90,52.41,0.0,Nov 25 2015 12:00AM,3.03,0.34,0.26,0.55,2.80,...,13.92,Semi-detached house,NO,1.0,Natural vent.,95.0,Masonry,NO,Y2002.0,NaN
4,NaN,110.66,173.21,0.0,May 30 2016 12:00AM,3.01,0.41,0.26,0.55,3.10,...,42.86,Detached house,NO,0.0,Natural vent.,144.0,Masonry,NO,Y2002.0,NaN


In [8]:
cats = []
for col in df.columns.values:
    if df[col].dtype == 'object':
        cats.append(col)

In [9]:
df_cont = df.drop(cats, axis=1)
df_cat = df[cats]

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from scipy.stats import skew

for col in df_cont.columns.values:
    if np.sum(df_cont[col].isnull()) > 50:
        #print("Removing Column: {}".format(col))
        df_cont = df_cont.drop(col, axis = 1)
    elif np.sum(df_cont[col].isnull()) > 0:
        #print("Replacing with Median: {}".format(col))
        median = df_cont[col].median()
        idx = np.where(df_cont[col].isnull())[0]
        df_cont[col].iloc[idx] = median
        
        
        outliers = np.where(is_outlier(df_cont[col]))
        df_cont[col].iloc[outliers] = median
        
               
        if skew(df_cont[col]) > 0.75:
            #print("Skewness Detected: {}".format(col))
            df_cont[col] = np.log(df_cont[col])
            df_cont[col] = df_cont[col].apply(lambda x: 0 if x == -np.inf else x)
        
        df_cont[col] = Normalizer().fit_transform(df_cont[col].reshape(1,-1))[0]
        

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: divide by zero encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: invalid value encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: RuntimeWarning: invalid value encountered in greater


In [ ]:
for col in df_cat.columns.values:
    if np.sum(df_cat[col].isnull()) > 50:
        df_cat = df_cat.drop(col, axis = 1)
        continue
    elif np.sum(df_cat[col].isnull()) > 0:
        df_cat[col] = df_cat[col].fillna('MIA')
        
    df_cat[col] = LabelEncoder().fit_transform(df_cat[col])
    
    num_cols = df_cat[col].max()
    for i in range(num_cols):
        col_name = col + '_' + str(i)
        df_cat[col_name] = df_cat[col].apply(lambda x: 1 if x == i else 0)
        
    df_cat = df_cat.drop(col, axis = 1)

##### Merge Numeric and Categorical Datasets and Create Training and Testing Data

In [11]:
target = train_1['EnergyRatingCat']

In [12]:
target = train_1[pd.notnull(train_1['EnergyRatingCat'])]
target = train_1['EnergyRatingCat']
target = target.dropna()
target = target.values


In [13]:
train_1.head()

,BuildingID,County,Type,Year,RatingType,EnergyRatingCat,EnergyRatingCont,GroundFloorArea,AvgWallU,AvgRoofU,...,EffSecondHeatGenPlant,PercSecondHeat,FuelTypeThirdBoiler,EffThirdHeatGenPlant,PercThirdHeat,HeatingSystemSolarSpace,TotPrimaryEnergyFac,TotCO2Emissions,PrimaryEnergySecondarySpace,EnergyMainWater
0,190835,Co. Wexford,Mid-terrace house,Y1890.0,Existing,F,418.42,52.04 sq. m,2.10,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,2335.6809
1,523113,Co. Clare,Detached house,Y1901.0,Existing,G,520.35,147.44 sq. m,2.01,0.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7144.838,4470.0759
2,271393,Co. Kerry,Detached house,Y1968.0,Existing,D2,281.33,104.84 sq. m,0.38,0.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2127.347,5837.4729
3,585961,Co. Cork,Semi-detached house,Y2002.0,Existing,C2,188.17,95.31 sq. m,0.55,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2771.886,3769.6248
4,624083,Co. Laois,Detached house,Y2002.0,Existing,B3,134.95,283.87 sq. m,0.55,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,5296.5828


In [14]:
target = train_1['EnergyRatingCat'].replace({"Nan" : -1, "A1" : 15 , "A3" : 13, "A2" : 14,"B1" : 12, "B2" : 11, "B3" : 10, "C1" : 9 , "C2" : 8, "C3" : 7,"D1" : 6, "D2" : 5,"D3" : 4, "E1" : 3 , "E2" : 2 , "F " : 1 ,"G " : 0})

In [56]:
target = train_1['EnergyRatingCat'].replace({"Nan" : -1, "A1" : 15 , "A3" : 13, "A2" : 14,"B1" : 12, "B2" : 11, "B3" : 10, "C1" : 9 , "C2" : 8, "C3" : 7,"D1" : 6, "D2" : 5,"D3" : 4, "E1" : 3 , "E2" : 2 , "F" : 1 ,"G" : 0})

In [15]:
train = df_cont[:train_1.shape[0]]
test = df_cont[train_1.shape[0]:]

In [53]:
# sk learn import 
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier


In [16]:
print (str(len(train))+" rows for training set")
print (str(len(test))+" rows for test set")
print (str(len(target))+" rows for test set")

464514 rows for training set
159262 rows for test set
464514 rows for test set


In [17]:
train = df_cont[:train_1.shape[0]]
test = df_cont[train_1.shape[0]:]

In [32]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
target = class_le.fit_transform(train[target].values)

KeyError: "['F ' 'G ' 'D2' ..., 'C2' 'B3' 'D1'] not in index"

In [21]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

In [18]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train, target)
#predictions = model.predict(test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [19]:
df_cont = df_cont.drop('BuildingID',axis =1)

In [20]:
df_cont = df_cont.dropna()

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train, target)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
df_cont.columns

Index(['AreaFirstFloor', 'AreaGroundFloor', 'AreaSecondFloor', 'AvgDoorU',
       'AvgFloorU', 'AvgRoofU', 'AvgWindowU', 'BridgingFac', 'BuildingReg',
       'ControlCat', 'DistLoss', 'EnergyMainWater', 'ExposedRoofArea',
       'ExposedWallArea', 'HeightFirstFloor', 'HeightGroundFloor',
       'LightingCO2', 'LightingEnergy', 'MainWaterEnergy', 'NCHPumps',
       'NChimneys', 'NFansVents', 'NGasFans', 'NOpenFlues', 'NShelteredSides',
       'NStoreys', 'PercDraughtStripped', 'PermTestResult',
       'PrimaryLightingEnergy', 'PrimaryPumpsFansEnergy', 'PumpsAndFansCO2',
       'PumpsAndFansEnergy', 'ResponseCat', 'RoofAreaRoom', 'SecWaterCO2',
       'SecWaterEnergy', 'SecWaterPrimaryEnergy', 'TempAdj', 'TotDoorArea',
       'TotFloorArea', 'TotWindowArea', 'UpgradeHES'],
      dtype='object')

In [23]:
df = df[np.isfinite(df['AreaFirstFloor'])]

In [24]:
df.dropna(how='any',axis = 0)

,AreaAperture,AreaFirstFloor,AreaGroundFloor,AreaSecondFloor,AssessmentDate,AvgDoorU,AvgFloorU,AvgRoofU,AvgWallU,AvgWindowU,...,TotWindowArea,Type,Underfloor,UpgradeHES,VentMethod,VolWaterStorage,WallStructure,WarmAir,Year,ZeroLossCollEff


In [25]:
import re
missing_values = []
nonumeric_values = []

print ("TRAIN SET INFORMATION")
print ("========================\n")

for column in train:
    # Find all the unique feature values
    uniq = train[column].unique()
    print ("'{}' has {} unique values" .format(column,uniq.size))
    if (uniq.size > 17):
        print("~~Listing up to 17 unique values~~")
    print (uniq[0:17])
    print ("\n-----------------------------------------------------------------------\n")
    
    # Find features with missing values
    if (True in pd.isnull(uniq)):
        s = "{} has {} missing" .format(column, pd.isnull(train[column]).sum())
        missing_values.append(s)
    
    # Find features with non-numeric values
    for i in range (1, np.prod(uniq.shape)):
        if (re.match('nan', str(uniq[i]))):
            break
        if not (re.search('(^\d+\.?\d*$)|(^\d*\.?\d+$)', str(uniq[i]))):
            nonumeric_values.append(column)
            break
  
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print ("Features with missing values:\n{}\n\n" .format(missing_values))
print ("Features with non-numeric values:\n{}" .format(nonumeric_values))
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

TRAIN SET INFORMATION

'AreaFirstFloor' has 13991 unique values
~~Listing up to 17 unique values~~
[ 0.          0.00141651  0.0011993   0.00309358  0.00116547  0.00168517
  0.00211988  0.00097398  0.00157978  0.00131867  0.00075732  0.00011182
  0.00125856  0.00115988  0.00186464  0.00104219  0.00164128]

-----------------------------------------------------------------------

'AreaGroundFloor' has 15108 unique values
~~Listing up to 17 unique values~~
[ 0.00108386  0.00140901  0.00143369  0.00122004  0.00126503  0.00140137
  0.00133517  0.0012038   0.00148939  0.00128343  0.00135037  0.00136858
  0.0010942   0.00123146  0.00121833  0.0011396   0.00132875]

-----------------------------------------------------------------------

'AreaSecondFloor' has 1 unique values
[ 0.]

-----------------------------------------------------------------------

'AvgDoorU' has 1 unique values
[ 0.00126615]

-----------------------------------------------------------------------

'AvgFloorU' has 112 uni

In [26]:
train = df[:train_1.shape[0]]
test = df[train_1.shape[0]:]

In [27]:
train

,AreaAperture,AreaFirstFloor,AreaGroundFloor,AreaSecondFloor,AssessmentDate,AvgDoorU,AvgFloorU,AvgRoofU,AvgWallU,AvgWindowU,...,TotWindowArea,Type,Underfloor,UpgradeHES,VentMethod,VolWaterStorage,WallStructure,WarmAir,Year,ZeroLossCollEff
0,NaN,0.00,33.69,0.00,Jun 17 2011 12:00AM,3.00,0.61,2.30,2.10,2.71,...,7.08,Mid-terrace house,NO,0.0,Natural vent.,110.0,Masonry,NO,Y1890.0,NaN
1,NaN,50.67,96.77,0.00,Apr 14 2015 12:00AM,3.96,0.94,0.75,2.01,2.63,...,37.00,Detached house,NO,0.0,Natural vent.,95.0,Masonry,NO,Y1901.0,NaN
2,NaN,0.00,104.84,0.00,Jun 28 2012 12:00AM,0.00,0.68,0.13,0.38,2.81,...,17.90,Detached house,NO,0.0,Natural vent.,110.0,Masonry,NO,Y1968.0,NaN
3,NaN,42.90,52.41,0.00,Nov 25 2015 12:00AM,3.03,0.34,0.26,0.55,2.80,...,13.92,Semi-detached house,NO,1.0,Natural vent.,95.0,Masonry,NO,Y2002.0,NaN
4,NaN,110.66,173.21,0.00,May 30 2016 12:00AM,3.01,0.41,0.26,0.55,3.10,...,42.86,Detached house,NO,0.0,Natural vent.,144.0,Masonry,NO,Y2002.0,NaN
5,NaN,0.00,94.40,0.00,May 13 2009 12:00AM,3.00,0.24,0.00,0.34,1.82,...,14.15,Ground-floor apartment,NO,0.0,Natural vent.,0.0,Masonry,NO,Y2007.0,NaN
6,NaN,0.00,76.15,0.00,Jun 29 2013 12:00AM,3.00,0.00,0.00,0.55,2.70,...,11.65,Mid-floor apartment,NO,0.0,Natural vent.,NaN,Masonry,NO,Y2003.0,NaN
7,NaN,41.69,49.72,0.00,Jan 21 2016 12:00AM,3.11,0.44,0.15,0.55,3.40,...,7.20,End of terrace house,NO,0.0,Natural vent.,95.0,Masonry,NO,Y2000.0,NaN
8,NaN,0.00,125.61,0.00,Sep 10 2013 12:00AM,3.00,0.57,0.13,0.31,4.80,...,19.60,Detached house,NO,0.0,Natural vent.,100.0,Masonry,NO,Y1989.0,NaN
9,NaN,60.28,64.38,0.00,Dec 12 2012 12:00AM,3.00,0.44,0.40,0.55,2.96,...,16.71,Semi-detached house,NO,0.0,Natural vent.,120.0,Insulated Conctete Form,NO,Y2002.0,NaN


In [28]:
target.isnull().values.any()

True

In [29]:
target = target.dropna()

In [30]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train, target)

ValueError: could not convert string to float: 'Y2004.0'

In [94]:
import re
missing_values = []
nonumeric_values = []

print ("TRAIN SET INFORMATION")
print ("========================\n")

for column in train:
    # Find all the unique feature values
    uniq = train[column].unique()
    print ("'{}' has {} unique values" .format(column,uniq.size))
    if (uniq.size > 17):
        print("~~Listing up to 17 unique values~~")
    print (uniq[0:17])
    print ("\n-----------------------------------------------------------------------\n")
    
    # Find features with missing values
    if (True in pd.isnull(uniq)):
        s = "{} has {} missing" .format(column, pd.isnull(train[column]).sum())
        missing_values.append(s)
    
    # Find features with non-numeric values
    for i in range (1, np.prod(uniq.shape)):
        if (re.match('nan', str(uniq[i]))):
            break
        if not (re.search('(^\d+\.?\d*$)|(^\d*\.?\d+$)', str(uniq[i]))):
            nonumeric_values.append(column)
            break
  
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print ("Features with missing values:\n{}\n\n" .format(missing_values))
print ("Features with non-numeric values:\n{}" .format(nonumeric_values))
print ("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

TRAIN SET INFORMATION

'AreaFirstFloor' has 14998 unique values
~~Listing up to 17 unique values~~
[   0.     50.67   42.9   110.66   41.69   60.28   75.83   34.84   56.51
   47.17   27.09    4.     45.02   41.49   66.7    37.28   58.71]

-----------------------------------------------------------------------

'AreaGroundFloor' has 22629 unique values
~~Listing up to 17 unique values~~
[  33.69   96.77  104.84   52.41  173.21   94.4    76.15   49.72  125.61
   64.38   80.     84.87   34.84   54.39   52.12  181.6    40.37]

-----------------------------------------------------------------------

'AreaSecondFloor' has 6553 unique values
~~Listing up to 17 unique values~~
[  0.    48.49  30.5   84.72  48.25  23.    52.44  11.76  59.74  20.83
  15.45  52.93  47.8   37.64  44.27  32.15  60.9 ]

-----------------------------------------------------------------------

'AvgDoorU' has 549 unique values
~~Listing up to 17 unique values~~
[ 3.    3.96  0.    3.03  3.01  3.11  1.31  2.85  4.69  1.

In [31]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train, target)

ValueError: could not convert string to float: 'Y2004.0'